## Training using ResNet152

### Importing Libraries and Setting GPU

In [1]:
import os
import torch
from PIL import Image
import numpy as np
from torchvision import models, transforms
from torch.utils.data import Dataset, DataLoader, random_split
from torch import nn, optim

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
# /content/drive/MyDrive/M9/cp/indfood-data

In [4]:
torch.__version__

'2.9.1+cu126'

In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [6]:
# models.resnet152(weights='IMAGENET1K_V2')
# models.convnext_small(weights='IMAGENET1K_V1')
# models.efficientnet_v2_s(weights='IMAGENET1K_V1')
# models.mobilenet_v2(weights='IMAGENET1K_V2')

### Data Loading & Model Definition

In [7]:
class FoodDataset(Dataset):
    def __init__(self, data_dir, transform=None):
        self.data_dir = data_dir
        self.transform = transform
        self.image_paths = []
        self.labels = []
        self.classes = sorted(os.listdir(data_dir))
        self.class_to_idx = {cls: i for i,cls in enumerate(self.classes)}

        for label_name in self.classes:
            label_dir = os.path.join(data_dir, label_name)
            for img_name in os.listdir(label_dir):
                self.image_paths.append(os.path.join(label_dir, img_name))
                self.labels.append(self.class_to_idx[label_name])


    def __len__(self):
        return len(self.image_paths)


    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert('RGB')
        label = self.labels[idx]

        if self.transform:
            image = self.transform(image)

        return image, label

In [8]:
input_size = 224

# ImageNet Normalization values
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]

# Simple transforms - just resize and normalize
train_transforms = transforms.Compose([
    # transforms.Resize((input_size, input_size))
    transforms.RandomRotation(15),           # Rotate up to 10 degrees
    transforms.RandomResizedCrop(224, scale=(0.9, 1.0)),  # Zoom
    transforms.RandomHorizontalFlip(),       # Horizontal flip
    transforms.ToTensor(),
    transforms.Normalize(mean=mean, std=std)
])

val_transforms = transforms.Compose([
    transforms.Resize((input_size, input_size)),
    transforms.ToTensor(),
    transforms.Normalize(mean=mean, std=std)
])

In [ ]:
train_dataset = FoodDataset(
    data_dir='../data/train',
    transform=train_transforms
)

val_dataset = FoodDataset(
    data_dir='../data/val',
    transform=val_transforms
)

In [10]:
total_train_len = len(train_dataset)
total_val_len = len(val_dataset)

# Define the length of the smaller dataset you want to use
subset_train_len = 25000
subset_val_len = 6000

# Define the lengths for the split: [desired_length, remaining_length]
train_lengths = [subset_train_len, total_train_len - subset_train_len]
val_lengths = [subset_val_len, total_val_len - subset_val_len]

# Randomly split the original dataset into two new datasets, You get a list of datasets; take the first one [0]
smaller_train_dataset = random_split(train_dataset, train_lengths)[0]
smaller_val_dataset = random_split(val_dataset, val_lengths)[0]



In [11]:
# Create the DataLoader with the smaller, randomly split dataset
train_loader = DataLoader(smaller_train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(smaller_val_dataset, batch_size=32, shuffle=False)

In [12]:
classes = list(train_dataset.class_to_idx)
classes

['aloo_gobi',
 'aloo_matar',
 'aloo_methi',
 'aloo_paratha',
 'aloo_shimla_mirch',
 'aloo_tikki',
 'amritsari_kulcha',
 'anda_curry',
 'ariselu',
 'balushahi',
 'banana_chips',
 'bandar_laddu',
 'basundi',
 'besan_laddu',
 'bhindi_masala',
 'biryani',
 'boondi',
 'boondi_laddu',
 'butter_chicken',
 'chaas',
 'chak_hao_kheer',
 'cham_cham',
 'chana_masala',
 'chapati',
 'chicken_pizza',
 'chicken_razala',
 'chicken_tikka',
 'chicken_tikka_masala',
 'chicken_wings',
 'chikki',
 'chivda',
 'chole_bhature',
 'daal_baati_churma',
 'daal_puri',
 'dabeli',
 'dal_khichdi',
 'dal_makhani',
 'dal_tadka',
 'dharwad_pedha',
 'dhokla',
 'double_ka_meetha',
 'dum_aloo',
 'falooda',
 'fish_curry',
 'gajar_ka_halwa',
 'garlic_bread',
 'gavvalu',
 'ghevar',
 'grilled_sandwich',
 'gujhia',
 'gulab_jamun',
 'hara_bhara_kabab',
 'idiyappam',
 'idli',
 'imarti',
 'jalebi',
 'kachori',
 'kadai_paneer',
 'kadhi_pakoda',
 'kaju_katli',
 'kakinada_khaja',
 'kalakand',
 'karela_bharta',
 'khakhra',
 'kheer',
 '

In [13]:
len(train_dataset), len(val_dataset)

(114964, 20370)

In [14]:
len(smaller_train_dataset), len(smaller_val_dataset)

(25000, 6000)

In [15]:
num_classes = len(classes)
num_classes

131

In [16]:
class FoodClassifierResNet(nn.Module):
    def __init__(self, num_classes=131, unfreeze_layers=0):
        super(FoodClassifierResNet, self).__init__()

        # load pre-trained ResNet-152
        self.base_model = models.resnet152(weights='IMAGENET1K_V2')

        # Freeze all base model parameters initially
        for param in self.base_model.parameters():
            param.requires_grad = False

        # Unfreeze specified number of 'layer' blocks from the end
        # For ResNet, layer4 is the last convolutional block, then layer3, etc.
        # unfreeze_layers=1 -> unfreeze layer4
        # unfreeze_layers=2 -> unfreeze layer4, layer3
        # unfreeze_layers=3 -> unfreeze layer4, layer3, layer2
        # unfreeze_layers=4 -> unfreeze layer4, layer3, layer2, layer1
        named_layer_blocks = ['layer4', 'layer3', 'layer2', 'layer1']

        for i in range(min(unfreeze_layers, len(named_layer_blocks))):
            layer_name = named_layer_blocks[i]
            if hasattr(self.base_model, layer_name):
                layer = getattr(self.base_model, layer_name)
                for param in layer.parameters():
                    param.requires_grad = True

        # Extract features (everything up to the adaptive average pooling, excluding the original FC layer)
        # The `base_model`'s `avgpool` is included in `features_extractor`.
        self.features_extractor = nn.Sequential(
            self.base_model.conv1,
            self.base_model.bn1,
            self.base_model.relu,
            self.base_model.maxpool,
            self.base_model.layer1,
            self.base_model.layer2,
            self.base_model.layer3,
            self.base_model.layer4,
            self.base_model.avgpool # Include the original avgpool
        )

        # add custom output layer
        # The input features to the linear layer will be from the base_model's final feature map size after avgpool
        self.output_layer = nn.Linear(self.base_model.fc.in_features, num_classes) # ResNet-152 has 2048 features before FC

    def forward(self, x):
        x = self.features_extractor(x)
        x = torch.flatten(x, 1) # Flatten the (batch_size, 2048, 1, 1) to (batch_size, 2048)
        x = self.output_layer(x)
        return x

In [17]:
model = FoodClassifierResNet(num_classes=131)
model.to(device)

FoodClassifierResNet(
  (base_model): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequen

### Tuning the Learning rate and Unfreezing layers

In [18]:
criterion = nn.CrossEntropyLoss()

In [19]:
# tuning the learning rate and unfreezing layers
def make_model(learning_rate=0.01, unfreeze_layers=0, scheduler_step_size=5, scheduler_gamma=0.1):
    model = FoodClassifierResNet(num_classes=131, unfreeze_layers=unfreeze_layers)
    model.to(device)

    # Only optimize parameters that require gradients (unfrozen layers and new head)
    optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=learning_rate)

    # Add a learning rate scheduler
    # StepLR decreases the learning rate by a factor of gamma every step_size epochs
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=scheduler_step_size, gamma=scheduler_gamma)

    return model, optimizer, scheduler

In [20]:
def train_and_evaluate(model, optimizer, scheduler, train_loader, val_loader, criterion, num_epochs, device):
    best_val_accuracy = 0.0  # Initialize variable to track the best validation accuracy

    for epoch in range(num_epochs):
        # Training phase
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0

        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            # forward pass
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            # backpropagation
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)

            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        train_loss = running_loss / len(train_loader)
        train_acc = correct / total

        # Validation phase
        model.eval()
        val_loss = 0.0
        val_correct = 0
        val_total = 0

        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)

                outputs = model(inputs)
                loss = criterion(outputs, labels)

                val_loss += loss.item()
                _, predicted = torch.max(outputs.data, 1)
                val_total += labels.size(0)
                val_correct += (predicted == labels).sum().item()

        val_loss /= len(val_loader)
        val_acc = val_correct / val_total

        print(f'Epoch {epoch+1}/{num_epochs}')
        print(f'  Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}')
        print(f'  Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}')

        # Step the learning rate scheduler after each epoch
        scheduler.step()

        if val_acc > best_val_accuracy:
            best_val_accuracy = val_acc
            checkpoint_path = f'food_resnet_v42_{epoch+1:02d}_{val_acc:.3f}.pth'
            torch.save(model.state_dict(), checkpoint_path)
            print(f'Checkpoint saved: {checkpoint_path}')

In [21]:
num_epochs = 15
# Iterate over different learning rates and also different numbers of unfrozen layers
# Example: Try unfreezing the last 1 or 2 convolutional blocks (layer4, layer3)
for lr in [0.001, 0.0001]: # Smaller learning rates are often better for fine-tuning unfrozen layers
    for unfreeze in [0, 1, 2]: # 0: freeze all base layers, 1: unfreeze layer4, 2: unfreeze layer4 & layer3
        print(f"--- Training with Learning Rate: {lr}, Unfrozen Layers: {unfreeze} ---")
        model, optimizer, scheduler = make_model(learning_rate=lr, unfreeze_layers=unfreeze)
        train_and_evaluate(model, optimizer, scheduler, train_loader, val_loader, criterion, num_epochs, device)
        print()
    print()

--- Training with Learning Rate: 0.001, Unfrozen Layers: 2 ---
Epoch 1/14
  Train Loss: 1.2552, Train Acc: 0.6817
  Val Loss: 0.8083, Val Acc: 0.7757
Checkpoint saved: food_resnet_v42_01_0.776.pth
Epoch 2/14
  Train Loss: 0.6511, Train Acc: 0.8107
  Val Loss: 0.6906, Val Acc: 0.8095
Checkpoint saved: food_resnet_v42_02_0.809.pth
Epoch 3/14
  Train Loss: 0.4858, Train Acc: 0.8541
  Val Loss: 0.6814, Val Acc: 0.8132
Checkpoint saved: food_resnet_v42_03_0.813.pth
Epoch 4/14
  Train Loss: 0.3938, Train Acc: 0.8794
  Val Loss: 0.5949, Val Acc: 0.8382
Checkpoint saved: food_resnet_v42_04_0.838.pth
Epoch 5/14
  Train Loss: 0.3119, Train Acc: 0.9030
  Val Loss: 0.5619, Val Acc: 0.8473
Checkpoint saved: food_resnet_v42_05_0.847.pth
Epoch 6/14
  Train Loss: 0.1426, Train Acc: 0.9577
  Val Loss: 0.4562, Val Acc: 0.8815
Checkpoint saved: food_resnet_v42_06_0.881.pth
Epoch 7/14
  Train Loss: 0.0957, Train Acc: 0.9716
  Val Loss: 0.4633, Val Acc: 0.8827
Checkpoint saved: food_resnet_v42_07_0.883.pth

In [22]:
# best lr: 0.001, unfrozen layers: 2

In [23]:
# # final training
# model, optimizer, scheduler = make_model(learning_rate=xxxxx, unfreeze_layers=yyyyy)
# train_and_evaluate(model, optimizer, scheduler, train_loader, val_loader, criterion, num_epochs=50, device)

In [24]:
# !pip install torchsummary

### Model Summary

In [28]:
from torchsummary import summary
summary(model, input_size=(3, 224, 224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]           9,408
            Conv2d-2         [-1, 64, 112, 112]           9,408
       BatchNorm2d-3         [-1, 64, 112, 112]             128
       BatchNorm2d-4         [-1, 64, 112, 112]             128
              ReLU-5         [-1, 64, 112, 112]               0
              ReLU-6         [-1, 64, 112, 112]               0
         MaxPool2d-7           [-1, 64, 56, 56]               0
         MaxPool2d-8           [-1, 64, 56, 56]               0
            Conv2d-9           [-1, 64, 56, 56]           4,096
           Conv2d-10           [-1, 64, 56, 56]           4,096
      BatchNorm2d-11           [-1, 64, 56, 56]             128
      BatchNorm2d-12           [-1, 64, 56, 56]             128
             ReLU-13           [-1, 64, 56, 56]               0
             ReLU-14           [-1, 64,

### Exporting to ONNX for serverles model deployment

In [29]:
## Exporting to ONNX for serverles model deployment
dummy_input = torch.randn(1, 3, 224, 224).to(device)

# Export to ONNX
onnx_path = "food_classifier_resnet152_v2.onnx"

torch.onnx.export(
    model,
    dummy_input,
    onnx_path,
    verbose=True,
    input_names=['input'],
    output_names=['output'],
    dynamic_axes={
        'input': {0: 'batch_size'},
        'output': {0: 'batch_size'}
    },
)

[torch.onnx] Obtain model graph for `FoodClassifierResNet([...]` with `torch.export.export(..., strict=False)`...
[torch.onnx] Obtain model graph for `FoodClassifierResNet([...]` with `torch.export.export(..., strict=False)`... ✅
[torch.onnx] Run decomposition...
[torch.onnx] Run decomposition... ✅
[torch.onnx] Translate the graph into ONNX...
[torch.onnx] Translate the graph into ONNX... ✅
Applied 310 of general pattern rewrite rules.


ONNXProgram(
    model=
        <
            ir_version=10,
            opset_imports={'': 20},
            producer_name='pytorch',
            producer_version='2.9.1+cu126',
            domain=None,
            model_version=None,
        >
        graph(
            name=main_graph,
            inputs=(
                %"input"<FLOAT,[s77,3,224,224]>
            ),
            outputs=(
                %"output"<FLOAT,[1,131]>
            ),
            initializers=(
                %"features_extractor.0.weight"<FLOAT,[64,3,7,7]>{Tensor(...)},
                %"features_extractor.4.0.conv1.weight"<FLOAT,[64,64,1,1]>{Tensor(...)},
                %"features_extractor.4.0.conv2.weight"<FLOAT,[64,64,3,3]>{Tensor(...)},
                %"features_extractor.4.0.conv3.weight"<FLOAT,[256,64,1,1]>{Tensor(...)},
                %"features_extractor.4.0.downsample.0.weight"<FLOAT,[256,64,1,1]>{Tensor(...)},
                %"features_extractor.4.1.conv1.weight"<FLOAT,[64,256,1,1]>{Tensor(